In [23]:
import json
import numpy as np

In [24]:
gt = 'mediapipe/keypoints/hot.mp4/0.671612024307251.json'
# cmp = 'mediapipe/keypoints/hot.mp4/0.955897331237793.json'
cmp = 'mediapipe/keypoints/hot.mp4/1.4703404903411865.json'

In [25]:
with open(gt, 'r') as file:
    gt_data = json.load(file)

with open(cmp, 'r') as file:
    cmp_data = json.load(file)

In [26]:
gt_data['keypoints'][0]['left_hip']

[0.5148850679397583,
 0.5666829943656921,
 -0.011555502191185951,
 0.9987103343009949]

In [27]:
gt_w, gt_h = gt_data['frame size']
cmp_w, cmp_h = cmp_data['frame size']
gt_w, gt_h

(953.0, 540.0)

In [28]:
gt_minx, gt_maxx, gt_miny, gt_maxy = gt_w, 0, gt_h, 0
cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = gt_w, 0, gt_h, 0

for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = int(gt_kp[0]*gt_w), int(gt_kp[1]*gt_h)
    cmp_kpx, cmp_kpy = int(cmp_kp[0]*cmp_w), int(cmp_kp[1]*cmp_h)

    gt_minx, gt_maxx, gt_miny, gt_maxy = min(gt_minx, gt_kpx), max(gt_maxx, gt_kpx), min(gt_miny, gt_kpy), max(gt_maxy, gt_kpy)
    cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = min(cmp_minx, cmp_kpx), max(cmp_maxx, cmp_kpx), min(cmp_miny, cmp_kpy), max(cmp_maxy, cmp_kpy)

    bbox_gt = (gt_maxx - gt_minx)*(gt_maxy - gt_miny)
    bbox_cmp = (cmp_maxx - cmp_minx)*(cmp_maxy-cmp_miny)
    
print("bbox gt", gt_maxx - gt_minx, gt_maxy - gt_miny)
print("bbox cmp", cmp_maxx - cmp_minx, cmp_maxy-cmp_miny)

bbox gt 94 297
bbox cmp 95 294


# compute oks

https://github.com/ZheC/Realtime_Multi-Person_Pose_Estimation/issues/7

순서 참고

"keypoints": [ "nose", "left_eye", "right_eye", "left_ear", "right_ear", "left_shoulder", "right_shoulder", "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle" ]

In [37]:
sigma = { "nose":0.026, 
"left_eye":0.025, 
"right_eye":0.025, 
"left_ear":0.035, 
"right_ear":0.035, 
"left_shoulder":0.079, 
"right_shoulder":0.079, 
"left_elbow":0.072, 
"right_elbow":0.072, 
"left_wrist":0.062, 
"right_wrist":0.062, 
"left_hip":0.107, 
"right_hip":0.107, 
"left_knee":0.087, 
"right_knee":0.087, 
"left_ankle":0.089, 
"right_ankle":0.089
}
# kp_constant =[.26, .25, .25, .35, .35, .79, .79, .72, .72, .62,.62, 1.07, 1.07, .87, .87, .89, .89]

In [30]:
for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = gt_kp[0]*gt_w, gt_kp[1]*gt_h
    cmp_kpx, cmp_kpy = cmp_kp[0]*cmp_w, cmp_kp[1]*cmp_h

    kp_c = kp_constant[kp]

    dx = gt_kpx - cmp_kpx
    dy = gt_kpy - cmp_kpy

    oks = np.exp(-(dx**2+dy**2)/(2*(bbox_gt**2)*(kp_c**2)))
    print(f"{kp:15}: {dx:2.5f} {dy:2.5f} {kp_c} {oks}")

left_hip -0.9243589043617249 7.076010704040527 0.107 0.9999971466249349
left_knee -1.9859544038772583 -0.8978748321533203 0.087 0.9999995973996366
left_ankle 0.5502528548240662 5.695145130157471 0.089 0.9999973486580667
right_hip 0.707171767950058 5.595914125442505 0.107 0.9999982173894723
right_knee 3.213358521461487 3.7128102779388428 0.087 0.9999979565168327
right_ankle 0.9611673951148987 4.3868279457092285 0.089 0.9999983666243999
left_shoulder -0.3765485882759094 3.0302953720092773 0.079 0.9999990415453307
left_elbow 0.10661965608596802 -1.781185269355774 0.072 0.9999996059894598
left_wrist -2.5385361909866333 0.7834678888320923 0.062 0.9999988221264356
right_shoulder 1.1402679681777954 0.8898603916168213 0.079 0.9999997849587204
right_elbow 0.9507440030574799 -1.9098669290542603 0.072 0.9999994367627256
right_wrist 1.5917968153953552 0.016543865203857422 0.062 0.9999995770976484
nose -0.7029399275779724 2.448797821998596 0.026 0.9999938404744013
left_eye -0.47981685400009155 2.32

In [45]:
gt = 'mediapipe/keypoints/hot.mp4/0.671612024307251.json'
cmp = 'mediapipe/keypoints/hot.mp4/13.356690645217896.json'

with open(gt, 'r') as file:
    gt_data = json.load(file)

with open(cmp, 'r') as file:
    cmp_data = json.load(file)

gt_minx, gt_maxx, gt_miny, gt_maxy = gt_w, 0, gt_h, 0
cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = gt_w, 0, gt_h, 0

for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = int(gt_kp[0]*gt_w), int(gt_kp[1]*gt_h)
    cmp_kpx, cmp_kpy = int(cmp_kp[0]*cmp_w), int(cmp_kp[1]*cmp_h)

    gt_minx, gt_maxx, gt_miny, gt_maxy = min(gt_minx, gt_kpx), max(gt_maxx, gt_kpx), min(gt_miny, gt_kpy), max(gt_maxy, gt_kpy)
    cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = min(cmp_minx, cmp_kpx), max(cmp_maxx, cmp_kpx), min(cmp_miny, cmp_kpy), max(cmp_maxy, cmp_kpy)

    bbox_gt = (gt_maxx - gt_minx)*(gt_maxy - gt_miny)
    bbox_cmp = (cmp_maxx - cmp_minx)*(cmp_maxy-cmp_miny)
    
print("bbox gt: ", gt_maxx - gt_minx, gt_maxy - gt_miny)
print("bbox cmp: ", cmp_maxx - cmp_minx, cmp_maxy-cmp_miny)

tot = 0

for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = gt_kp[0]*gt_w, gt_kp[1]*gt_h
    cmp_kpx, cmp_kpy = cmp_kp[0]*cmp_w, cmp_kp[1]*cmp_h

    sig = sigma[kp]
    d = (dx**2+dy**2)**0.5
    s = bbox_cmp
    k = 2*sig

    dx = gt_kpx - cmp_kpx
    dy = gt_kpy - cmp_kpy

    oks = np.exp(-d**2/(2*(s**2)*(k**2)))*sig
    tot += oks
    print(f"{kp:15}: {dx:2.5f} {dy:2.5f} {kp_c} {oks}")

print(tot)

bbox gt:  94 297
bbox cmp:  95 294
left_hip       : -0.92436 7.07601 0.035 0.10699999583725013
left_knee      : -1.98595 -0.89787 0.035 0.08699990620616507
left_ankle     : 0.55025 5.69515 0.035 0.08899999144757112
right_hip      : 0.70717 5.59591 0.035 0.10699995097369129
right_knee     : 3.21336 3.71281 0.035 0.08699994140348953
right_ankle    : 0.96117 4.38683 0.035 0.08899995659031402
left_shoulder  : -0.37655 3.03030 0.035 0.07899995909221441
left_elbow     : 0.10662 -1.78119 0.035 0.07199997924799074
left_wrist     : -2.53854 0.78347 0.035 0.061999991770983316
right_shoulder : 1.14027 0.88986 0.035 0.07899998568399234
right_elbow    : 0.95074 -1.90987 0.035 0.07199999534402866
right_wrist    : 1.59180 0.01654 0.035 0.06199998823663654
nose           : -0.70294 2.44880 0.035 0.025999984382308654
left_eye       : -0.47982 2.32908 0.035 0.02499995839728526
right_eye      : -0.26845 1.59652 0.035 0.024999963755007636
left_ear       : -0.62131 2.98852 0.035 0.03499998800072564
right_e

In [44]:
gt = 'mediapipe/keypoints/hot.mp4/0.671612024307251.json'
cmp = 'mediapipe/keypoints/hot.mp4/0.955897331237793.json'

with open(gt, 'r') as file:
    gt_data = json.load(file)

with open(cmp, 'r') as file:
    cmp_data = json.load(file)

gt_minx, gt_maxx, gt_miny, gt_maxy = gt_w, 0, gt_h, 0
cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = gt_w, 0, gt_h, 0

for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = int(gt_kp[0]*gt_w), int(gt_kp[1]*gt_h)
    cmp_kpx, cmp_kpy = int(cmp_kp[0]*cmp_w), int(cmp_kp[1]*cmp_h)

    gt_minx, gt_maxx, gt_miny, gt_maxy = min(gt_minx, gt_kpx), max(gt_maxx, gt_kpx), min(gt_miny, gt_kpy), max(gt_maxy, gt_kpy)
    cmp_minx, cmp_maxx, cmp_miny, cmp_maxy = min(cmp_minx, cmp_kpx), max(cmp_maxx, cmp_kpx), min(cmp_miny, cmp_kpy), max(cmp_maxy, cmp_kpy)

    bbox_gt = (gt_maxx - gt_minx)*(gt_maxy - gt_miny)
    bbox_cmp = (cmp_maxx - cmp_minx)*(cmp_maxy-cmp_miny)
    
print("bbox gt: ", gt_maxx - gt_minx, gt_maxy - gt_miny)
print("bbox cmp: ", cmp_maxx - cmp_minx, cmp_maxy-cmp_miny)

tot = 0

for kp in gt_data['keypoints'][0]:
    gt_kp = gt_data['keypoints'][0][kp]
    cmp_kp = cmp_data['keypoints'][0][kp]
    gt_kpx, gt_kpy = gt_kp[0]*gt_w, gt_kp[1]*gt_h
    cmp_kpx, cmp_kpy = cmp_kp[0]*cmp_w, cmp_kp[1]*cmp_h

    sig = sigma[kp]
    d = (dx**2+dy**2)**0.5
    s = bbox_cmp
    k = 2*sig

    dx = gt_kpx - cmp_kpx
    dy = gt_kpy - cmp_kpy

    oks = np.exp(-d**2/(2*(s**2)*(k**2)))*sig
    tot += oks
    print(f"{kp:15}: {dx:2.5f} {dy:2.5f} {kp_c} {oks}")

print(tot)

bbox gt:  94 297
bbox cmp:  97 293
left_hip       : 0.15473 4.94372 0.035 0.10699999656294686
left_knee      : 0.28970 3.85833 0.035 0.08699995648442642
left_ankle     : 1.32823 4.94748 0.035 0.08899997396962485
right_hip      : 1.16398 4.22132 0.035 0.1069999620474409
right_knee     : 4.32591 4.89789 0.035 0.08699996589369408
right_ankle    : 1.57521 3.98154 0.035 0.088999925749918
left_shoulder  : 0.17399 1.55771 0.035 0.0789999640863743
left_elbow     : 0.75883 -1.45832 0.035 0.07199999471969473
left_wrist     : 1.00144 0.47860 0.035 0.0619999932545533
right_shoulder : 0.63103 1.04613 0.035 0.0789999975867991
right_elbow    : 2.99762 -0.58530 0.035 0.07199999679199662
right_wrist    : 1.69197 0.42972 0.035 0.061999976716844436
nose           : 0.02531 1.94165 0.035 0.02599998186196285
left_eye       : -0.22099 1.77929 0.035 0.024999976659656375
right_eye      : 0.12497 1.67161 0.035 0.024999980101000796
left_ear       : -0.22937 1.98750 0.035 0.034999987576280533
right_ear      : 0.